### Convert ROMS.nc mesh to shapefile

In [133]:
import ocsmesh
import xarray as xr
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, mapping

In [134]:
path_roms = r"C:/Users/Felicio.Cassalho/Work/Modeling/AK_Project/ROMS2SCHISM/inputs_roms/Dsm4_his_0366.nc"

In [135]:
roms = xr.open_dataset(path_roms)

In [136]:
# #loading the center of the cells, mask indicates if it is wet or dry
# lats = roms.variables["lat_rho"][:]
# lons = roms.variables["lon_rho"][:]
mask_rho =  np.array(roms.variables["mask_rho"][:])

In [137]:
#loading the vertices of the cells, mask indicates if it is wet or dry
lats_psi = np.array(roms.variables["lat_psi"][:])
lons_psi = np.array(roms.variables["lon_psi"][:])+360
mask_psi =  np.array(roms.variables["mask_psi"][:])

In [138]:
# #loading the u and v of the cells, mask indicates if it is wet or dry
# lat_u =  roms.variables["lat_u"][:]
# lon_u =  roms.variables["lon_u"][:]
# mask_u = roms.variables["mask_u"][:]
# lat_v =  roms.variables["lat_v"][:]
# lon_v =  roms.variables["lon_v"][:]
# mask_v = roms.variables["mask_v"][:]

In [139]:
poly=[]
wetdry=[]
for i in range(len(mask_psi)):
    # print(i,len(mask_psi))
    for j in range(len(mask_psi[i])):
        # print(i,j, mask_psi.shape)
        if i < len(mask_psi)-1 and j < len(mask_psi[i])-1:
            p = [(lons_psi[i][j],lats_psi[i][j]),
                 (lons_psi[i][j+1],lats_psi[i][j+1]),
                 (lons_psi[i+1][j+1],lats_psi[i+1][j+1]),
                 (lons_psi[i+1][j],lats_psi[i+1][j]),
                 (lons_psi[i][j],lats_psi[i][j]),
                ]
            poly.append(p)
        if i < len(mask_psi)-1 and j < len(mask_psi[i])-1:
            wetdry.append(mask_rho[i+1][j+1])



In [140]:
# To determine if the polygon is wet or dry we adopted the criteria:
# if the rho and the psi vertices of the polygon are wet, then the whole polygon is wet
# maybe the correct when it to consider wet if all vertices (rho, psi, u, and v) are wet, I don't know,
wetdry_final=[]
for i in range(len(wetdry)):
    if wetdry[i] == 1:
        wetdry_final.append(1)

    else:
        wetdry_final.append(0)    

In [141]:
# This loop drops all polygon that are considered dry based on the criteria above
poly_final = []
for i in range(len(wetdry_final)):
    if wetdry_final[i] == 1:
        poly_final.append(poly[i])
               

In [142]:
polygon = [Polygon(i) for i in poly_final]

In [143]:
gdf = gpd.GeoDataFrame(
        geometry = polygon,crs=4326)

In [144]:
gdf.to_file(r'C:\Users\Felicio.Cassalho\Work\Modeling\AK_Project\AK_Paper\Fig1\ak_roms_1.shp')